In [2]:
import torch
from openmmtools.constants import kB
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
from openmmtools.testsystems import AlanineDipeptideVacuum
import numpy as np
import mdtraj

import sys
# sys.path.insert(1, '/var/home/vs488/Documents/boltzmann/code/boltzmann-generators/')
sys.path.append("../")
import boltzgen.zmatrix as zmatrix
import boltzgen.internal as ics

In [6]:
# Generate a trajectory for use as training data
temperature = 298
kT = kB * temperature
testsystem = AlanineDipeptideVacuum()
vacuum_sim = app.Simulation(testsystem.topology,
                            testsystem.system,
                            mm.LangevinIntegrator(temperature * unit.kelvin , 1.0 / unit.picosecond, 1.0 * unit.femtosecond),
                            platform=mm.Platform.getPlatformByName('CPU')
                            )
vacuum_sim.context.setPositions(testsystem.positions)
vacuum_sim.minimizeEnergy()
vacuum_sim.reporters.append(mdtraj.reporters.HDF5Reporter('aldp.h5', 1000))
vacuum_sim.step(10000)

In [3]:
aldp_traj = mdtraj.load('aldp.h5')

In [5]:
z = [
    (1, [4, 5, 6]),
    (0, [1, 4, 5]),
    (2, [1, 0, 4]),
    (3, [1, 0, 2]),
    (7, [6, 4, 5]),
    (9, [8, 6, 7]),
    (10, [8, 6, 9]),
    (11, [10, 8, 9]),
    (12, [10, 8, 11]),
    (13, [10, 11, 12]),
    (17, [16, 14, 15]),
    (19, [18, 16, 17]),
    (20, [18, 19, 16]),
    (21, [18, 19, 20])
]

backbone_indices = [4, 5, 6, 8, 14, 15, 16, 18]
# center everything
aldp_traj.center_coordinates()

# superpose on the backbone
ind = aldp_traj.top.select("backbone")

aldp_traj.superpose(aldp_traj, 0, atom_indices=ind, ref_atom_indices=ind)

# Gather the training data into a pytorch Tensor with the right shape
training_data = aldp_traj.xyz
n_atoms = training_data.shape[1]
n_dim = n_atoms * 3
training_data_npy = training_data.reshape(-1, n_dim)
training_data = torch.from_numpy(training_data_npy.astype("float32"))

ic_transform = ics.InternalCoordinateTransform(n_dim, z, backbone_indices, training_data)
transformed_c, jac = ic_transform.forward(training_data)
print(transformed_c)

tensor([[-2.2335e-04, -5.9222e-01, -3.7730e-01,  7.9655e-01,  3.8604e-01,
         -1.2845e-01,  9.6573e-04,  7.9758e-01,  2.2106e-02,  7.4461e-05,
         -3.0964e-01,  7.0399e-02, -1.7981e-01, -1.0579e-01, -8.4798e-02,
         -1.3757e-01, -2.2135e-01, -7.7773e-02, -9.9417e-02, -3.8581e-03,
         -6.7463e-02, -2.2342e-04, -1.6478e+00, -1.9516e-02,  4.6907e-02,
          7.3869e-04, -5.8410e-02, -3.7218e-04,  4.4527e-01,  4.5370e-01,
         -7.4636e-01,  5.0697e-01, -8.0521e-02, -2.9786e-04, -5.6710e-01,
          2.5496e-01,  4.4670e-04,  1.6050e+00, -9.3535e-02, -9.6637e-04,
         -1.1257e+00,  5.2443e-02,  9.6218e-02,  7.6031e-02,  6.5161e-02,
          9.9008e-02,  1.9980e-01,  7.2158e-02,  1.5522e-01, -1.7300e-03,
          1.5635e-01, -2.9788e-04, -5.4176e-01, -2.0081e-01,  2.1631e-01,
          3.6323e-02,  2.7869e-01, -4.4661e-04,  7.4852e-01, -4.3499e-01,
          8.9182e-04,  6.0778e-01, -2.7853e-02, -1.4892e-04, -3.2847e-01,
          1.2846e-02],
        [-7.445